In [1]:
import arcgis
from arcgis.gis import GIS
import csv
import shapefile

In [2]:
def process_file(fname):
    ''''''
    with open(fname,'r') as file:
        data = [ line.rstrip().split(',') for line in file]   
    return data

In [3]:
def remove_null_lat_long_rows(data):
    '''Returns a new list of list that excludeds any row with an empty string in lat/long col.'''
    new_data = []
    for row in range(len(data)):
        if data[row][-1] != '':
            new_data.append(data[row])
    return new_data

In [4]:
def output_csv(data, file_name):
    '''Outputs a new .csv file to the working directory.'''   
    with open(file_name, 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(data)

In [5]:
def create_shp(fname,header):
    '''Returns a .shp file created from fname.csv'''
    data = process_file(fname) # Covert .csv to list of list.
    RLCV_shp = shapefile.Writer(shapefile.POINT)
    RLCV_shp.autoBalance = 1
 
    # Create field names for shp attribute table. 
    RLCV_shp.field('obj_id','N','6')
    RLCV_shp.field(header[0],'C','50')
    RLCV_shp.field(header[1],'N','6')
    RLCV_shp.field(header[2],'C','50')
    RLCV_shp.field(header[3],'C','15')
    RLCV_shp.field(header[4],'N','3')
    RLCV_shp.field(header[5],'F',20,10)
    RLCV_shp.field(header[6],'F',20,10)
    RLCV_shp.field(header[7],'F',20,10)
    RLCV_shp.field(header[8],'F',20,10)
    RLCV_shp.field(header[9],'C','50')

    for lst in range(1,len(data)):
        row = data[lst]
        RLCV_shp.point(float(row[5]),float(row[6]))
        RLCV_shp.record(lst,row[0],row[1],row[2],row[3],row[4],row[5],row[6],row[7],row[8],row[9])
    
    RLCV_shp.save('RLCV_shp')

#### Main()
ddddddddddddd

In [6]:
fname = 'ESRI project shps/Red_Light_Camera_Violations.csv'
data = process_file(fname)
print('Total number of rows in the original file: {0:,}'.format(len(data)) )
clean_data = remove_null_lat_long_rows(data)
print('Total number of rows after removing all rows with no lat/long data: {0:,}'.format(len(clean_data)) )
print('Number of deleted rows: {0:,}'.format(len(data) - len(clean_data)) )

Total number of rows in the original file: 310,580
Total number of rows after removing all rows with no lat/long data: 291,981
Number of deleted rows: 18,599


In [ ]:
item = gis.content.add({}, data_dir + current_sd_file)   # .sd file is uploaded and a .sd file item is created
published_item = item.publish() 

#### Display the field names of the soon to be created shapefile.

In [7]:
headers = clean_data[0]
headers # Display column names.

['INTERSECTION',
 'CAMERA ID',
 'ADDRESS',
 'VIOLATION DATE',
 'VIOLATIONS',
 'X COORDINATE',
 'Y COORDINATE',
 'LATITUDE',
 'LONGITUDE',
 'LOCATION']

#### Create a new shapfile based on the cleaned .csv file.

In [8]:
output_csv(clean_data, 'clean_RLCV.csv')
create_shp('clean_RLCV.csv',headers)

#### Upload newly created shapfile to ArcGIS Online:

In [9]:
gis = GIS("https://www.arcgis.com","jesse.j.franks@gmail.com","ZEpar999!!") # Connect to ArcGIS Online.

RLCV_properties = {'title':'Red_Light_Camera_Violations_Chicago', 'tags':'Chicago, Traffic','type':'Shapefile'}
 
RLCV_shp = gis.content.add(RLCV_properties, data='./ESRI project shps/RLCV.zip')

In [10]:
RLCV_shp_feature_layer = RLCV_shp.publish()

Exception: Job failed.

#### Search for data.

In [ ]:
search = gis.content.search("Chicago", item_type="Feature Layer")
search

#### Get data.

In [ ]:
RLCV_shp = search[0]
RLCV_shp

In [ ]:
search_result = gis.content.search("title:Chicago crime","Feature Layer", outside_org=True)
search_result

In [ ]:
USA_Freeway_System = search_result[2]

In [ ]:
Chicago_crime = gis.map('Chicago, IL')
Chicago_crime.basemap = 'dark-gray-vector'
Chicago_crime.add_layer(USA_Freeway_System)

In [ ]:
update_parameters = {"extent": {"xmin": -87.83,"ymin": 41.69,"xmax": -87.43,"ymax": 42.08,
                                "spatialReference": {"wkid": 4326}}}
Chicago_crime.update(update_parameters)

In [ ]:
Chicago_crime